In [53]:
import pandas as pd
import numpy as np
import random
import transformers
import torch
import dotenv
import os
import matplotlib.pyplot as plt
import re
import string
from tqdm import tqdm

dotenv.load_dotenv()

True

In [54]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    token=os.getenv('HF_TOKEN')
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.31it/s]


# Rule 110 Cellular Automaton

In [55]:
def make_cellular_problem_set(size, steps, num_problems, boundary='wrap'):
    if steps < 3:
        raise ValueError("Need 3 steps to have a solution and 2 intermediates")
    def int_to_binary_list(n, min_length=8):
        binary = bin(n)[2:]  # Convert to binary string and remove '0b' prefix
        binary_list = [int(b) for b in binary.zfill(min_length)]  # Pad with zeros if necessary
        return binary_list
    
    dict_110 = {
        (0, 0, 0): 0,
        (0, 0, 1): 1,
        (0, 1, 0): 1,
        (0, 1, 1): 1,
        (1, 0, 0): 0,
        (1, 0, 1): 1,
        (1, 1, 0): 1,
        (1, 1, 1): 0
    }

    def rule_110(prev):
        next_state = []
        for i in range(len(prev)):
            left = prev[(i-1) % len(prev)] if boundary == 'wrap' or (i > 0 and i < len(prev)-1) else boundary
            center = prev[i]
            right = prev[(i+1) % len(prev)] if boundary == 'wrap' or (i > 0 and i < len(prev)-1) else boundary
            pattern = (left, center, right)
            next_state.append(dict_110[pattern])
        return next_state
    
    def make_rule_110_problem(initial_state, steps):
        current_state = initial_state
        states = [current_state]
        for _ in range(steps):
            current_state = rule_110(current_state)
            states.append(current_state)
        return (''.join(str(x) for x in initial_state),
            ''.join(str(x) for x in states[-1]),
            ''.join(str(x) for x in states[1]),
            ''.join(str(x) for x in states[-2]))

    return pd.DataFrame(
        [make_rule_110_problem(int_to_binary_list(((i+1)*33581)%(2**(size))), steps) for i in range(num_problems)]
        , columns=['problem', 'correct_solution', 'intermediate_1', 'intermediate_2'])

In [56]:
make_cellular_problem_set(10, 3, 1000, boundary=0)

,problem,correct_solution,intermediate_1,intermediate_2
0,1100101101,1001000011,1101111111,1111000001
1,1001011010,1010000110,1011111110,1110000010
2,110000111,110110001,110001101,110011111
3,10110100,10001100,11111100,10000100
4,1111100001,1011101101,1000100011,1001100111
...,...,...,...,...
995,1100010100,1111101100,1100111100,1101100100
996,1001000001,1001001101,1011000011,1111000111
997,101101110,100011010,111111010,100001110
998,10011011,10100011,10111111,11100001


# SAT

In [57]:
from pysat.formula import CNF
from pysat.solvers import Glucose3

def solve_nsat(clauses):
    # Create a CNF formula
    cnf = CNF()
    for clause in clauses:
        cnf.append(clause)

    # Create a SAT solver
    with Glucose3(bootstrap_with=cnf) as solver:
        # Check if the formula is satisfiable
        if solver.solve():
            return solver.get_model()
        else:
            return None

In [58]:
def make_nsat_problem_set(vars_per_clause, num_clauses, num_problems):
    def make_nsat_problem(vars_per_clause, num_clauses):
        text_variables = [string.ascii_lowercase[i] for i in range(vars_per_clause)]
        text_problem = []
        pysat_problem = []
        for _ in range(num_clauses):
            clause = random.sample(range(vars_per_clause), 3)
            signs = [random.choice([-1, 1]) for _ in range(3)]
            pysat_clause = [signs[i]*(var+1) for i, var in enumerate(clause)]
            pysat_problem.append(pysat_clause)
            pysat_solution = solve_nsat(pysat_problem)
            if pysat_solution is None:
                text_solution = None
            else:
                text_solution_letters = [f"{'¬' if var <0 else ''}{text_variables[abs(var)-1]}" for i, var in enumerate(pysat_solution)]
                text_solution = f"{' ^ '.join(text_solution_letters)}"

            text_clause = [f"{'¬' if signs[i] == -1 else ''}{text_variables[var]}" for i, var in enumerate(clause)]
            text_problem.append(f"({' v '.join(text_clause)})")
        return ' ^ '.join(text_problem), text_solution, None, None

    return pd.DataFrame(
        [make_nsat_problem(vars_per_clause, num_clauses) for _ in range(num_problems)],
        columns=['problem', 'correct_solution', 'intermediate_1', 'intermediate_2'])

In [59]:
make_nsat_problem_set(3, 20, 1000)

,problem,correct_solution,intermediate_1,intermediate_2
0,(¬b v c v a) ^ (b v ¬a v ¬c) ^ (c v b v ¬a) ^ ...,¬a ^ b ^ c,None,None
1,(c v ¬b v a) ^ (a v ¬c v ¬b) ^ (¬c v b v a) ^ ...,None,None,None
2,(c v ¬a v b) ^ (a v ¬b v c) ^ (c v ¬b v ¬a) ^ ...,None,None,None
3,(b v ¬a v c) ^ (c v ¬a v ¬b) ^ (c v b v ¬a) ^ ...,None,None,None
4,(¬a v b v ¬c) ^ (¬a v ¬c v ¬b) ^ (a v ¬c v b) ...,None,None,None
...,...,...,...,...
995,(¬a v c v b) ^ (a v c v ¬b) ^ (a v ¬b v c) ^ (...,None,None,None
996,(c v ¬a v b) ^ (¬c v ¬a v ¬b) ^ (c v ¬b v ¬a) ...,¬a ^ b ^ c,None,None
997,(a v c v ¬b) ^ (c v ¬b v ¬a) ^ (¬b v a v ¬c) ^...,None,None,None
998,(a v c v ¬b) ^ (¬c v ¬a v b) ^ (c v b v a) ^ (...,None,None,None


# Dot Product

In [60]:
def make_dot_product_problem_set(vec_len, vec_mag, num_problems):
    if vec_len < 2:
        raise ValueError("Need vectors of length 2 or greater to have two intermediates")
    def make_dot_product_problem(vec_len):
        a = np.random.randint(-vec_mag, vec_mag, vec_len)
        b = np.random.randint(-vec_mag, vec_mag, vec_len)
        return (f"[{', '.join([str(x) for x in a])}] ⋅ [{', '.join([str(x) for x in b])}]"
        , np.dot(a, b)
        , a[0]*b[0],
        a[-1]*b[-1])

    return pd.DataFrame(
        [make_dot_product_problem(vec_len) for _ in range(num_problems)],
        columns=['problem', 'correct_solution', 'intermediate_1', 'intermediate_2'])

In [61]:
make_dot_product_problem_set(3, 10, 1000)

,problem,correct_solution,intermediate_1,intermediate_2
0,"[0, 7, 5] ⋅ [1, -2, -9]",-59,0,-45
1,"[-3, -8, -7] ⋅ [9, -10, -3]",74,-27,21
2,"[-3, -3, -9] ⋅ [-6, 8, -9]",75,18,81
3,"[8, -6, 6] ⋅ [-9, -7, -2]",-42,-72,-12
4,"[-1, -1, 0] ⋅ [-6, 5, 1]",1,6,0
...,...,...,...,...
995,"[-5, -5, -2] ⋅ [3, 8, -1]",-53,-15,2
996,"[7, -10, 7] ⋅ [-7, 8, 2]",-115,-49,14
997,"[-4, -1, -6] ⋅ [2, -6, -3]",16,-8,18
998,"[-4, 8, 1] ⋅ [-1, 2, 4]",24,4,4


In [109]:
def solve_problem_cot(problem, sys_prompt, cot_prompt):
    messages = [
    {"role": "system", "content": sys_prompt + ' ' + cot_prompt},
    {"role": "user", "content": problem},
    ]

    outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=pipeline.tokenizer.eos_token_id
    )

    return outputs[0]['generated_text'][-1]['content']

def solve_problem_memo(problem, sys_prompt, memo_prompt, max_toks=10): # setting max_toks to 1 because we're doing small digit numbers
    messages = [
    {"role": "system", "content": sys_prompt + ' ' + memo_prompt},
    {"role": "user", "content": problem},
    ]
    print(messages)

    outputs = pipeline(
    messages,
    max_new_tokens=max_toks,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=pipeline.tokenizer.eos_token_id
    )

    return outputs[0]['generated_text'][-1]['content']

In [63]:
problem_prompt = "What is the dot product of these two vectors?"
cot_prompt = "Show your work."
memo_prompt = "Answer with only a number."
n=100

In [64]:
def dot_test(dot_problems, problem_prompt=problem_prompt, cot_prompt=cot_prompt, memo_prompt=memo_prompt, n=n):
    cot_solutions = []
    cot_correct = 0
    for i, row in tqdm(list(dot_problems.iterrows())[:n]):
        cot_solutions.append(solve_problem_cot(row['problem'], problem_prompt, cot_prompt))
        cot_correct += str(row['correct_solution']) in cot_solutions[-1]

    memo_solutions = []
    memo_correct = 0
    for i, row in tqdm(list(dot_problems.iterrows())[:n]):
        memo_solutions.append(solve_problem_memo(row['problem'], problem_prompt, memo_prompt))
        memo_correct += str(row['correct_solution']) in memo_solutions[-1]

    print(cot_correct, memo_correct)
    return cot_solutions, memo_solutions

In [65]:
results = {}

In [66]:
results[(3, 10)] = dot_test(make_dot_product_problem_set(3, 10, n))

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:17<00:00,  5.68it/s]

80 5


In [71]:
results[(2, 10)] = dot_test(make_dot_product_problem_set(2, 10, n))

100%|██████████| 100/100 [00:13<00:00,  7.35it/s]

87 26


In [67]:
results[(2, 20)] = dot_test(make_dot_product_problem_set(2, 20, n))

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:12<00:00,  8.13it/s]

90 19


In [69]:
results[(5, 10)] = dot_test(make_dot_product_problem_set(5, 10, n))

100%|██████████| 100/100 [00:16<00:00,  6.05it/s]

70 3


In [110]:
def repeated_solve_problem_memo(problem, sys_prompt, memo_prompt, max_toks=10):
    soln = solve_problem_memo(problem, sys_prompt, memo_prompt, max_toks=max_toks)
    i = 0
    while sum([len([n for n in soln if n.isnumeric()]) > 3]):
        i += 1
        print('retrying', i, soln)
        soln = solve_problem_memo(problem, sys_prompt, memo_prompt, max_toks=max_toks)
    return soln

def test_memo_prompt(memo_prompt):
    dot_problems = make_dot_product_problem_set(3, 10, n)
    memo_solutions = []
    memo_correct = 0
    for i, row in tqdm(list(dot_problems.iterrows())[:n]):
        memo_solutions.append(repeated_solve_problem_memo(row['problem'], problem_prompt, memo_prompt=memo_prompt, max_toks=20))
        memo_correct += str(row['correct_solution']) in memo_solutions[-1]
    print(sum([len([n for n in x if n.isnumeric()]) <= 2 for x in memo_solutions]))
    print('\n'.join(memo_solutions))

test_memo_prompt("Answer with only a number. Do not do any calculations.")

  0%|          | 0/100 [00:00<?, ?it/s]

  2%|▏         | 2/100 [00:00<00:08, 11.70it/s]

[{'role': 'system', 'content': 'What is the dot product of these two vectors? Answer with only a number. Do not do any calculations.'}, {'role': 'user', 'content': '[-1, -9, 6] ⋅ [-3, -5, -1]'}]
[{'role': 'system', 'content': 'What is the dot product of these two vectors? Answer with only a number. Do not do any calculations.'}, {'role': 'user', 'content': '[-5, -4, -3] ⋅ [6, 7, -6]'}]
[{'role': 'system', 'content': 'What is the dot product of these two vectors? Answer with only a number. Do not do any calculations.'}, {'role': 'user', 'content': '[-3, 6, 1] ⋅ [-2, 0, 5]'}]


  4%|▍         | 4/100 [00:00<00:07, 13.17it/s]

[{'role': 'system', 'content': 'What is the dot product of these two vectors? Answer with only a number. Do not do any calculations.'}, {'role': 'user', 'content': '[-6, -1, -1] ⋅ [-8, 1, 0]'}]
[{'role': 'system', 'content': 'What is the dot product of these two vectors? Answer with only a number. Do not do any calculations.'}, {'role': 'user', 'content': '[1, -7, 0] ⋅ [-3, -3, 0]'}]
[{'role': 'system', 'content': 'What is the dot product of these two vectors? Answer with only a number. Do not do any calculations.'}, {'role': 'user', 'content': '[8, -8, -10] ⋅ [-5, 4, -2]'}]


  8%|▊         | 8/100 [00:00<00:08, 11.31it/s]

[{'role': 'system', 'content': 'What is the dot product of these two vectors? Answer with only a number. Do not do any calculations.'}, {'role': 'user', 'content': '[5, -9, 7] ⋅ [-8, 6, -8]'}]
[{'role': 'system', 'content': 'What is the dot product of these two vectors? Answer with only a number. Do not do any calculations.'}, {'role': 'user', 'content': '[0, 4, 2] ⋅ [-10, -9, -2]'}]
[{'role': 'system', 'content': 'What is the dot product of these two vectors? Answer with only a number. Do not do any calculations.'}, {'role': 'user', 'content': '[7, -9, 5] ⋅ [-5, -9, -6]'}]
[{'role': 'system', 'content': 'What is the dot product of these two vectors? Answer with only a number. Do not do any calculations.'}, {'role': 'user', 'content': '[-4, 8, 6] ⋅ [1, 7, 9]'}]
retrying 1 -4*1 + 8*7 + 6*9 = 68
[{'role': 'system', 'content': 'What is the dot product of these two vectors? Answer with only a number. Do not do any calculations.'}, {'role': 'user', 'content': '[-4, 8, 6] ⋅ [1, 7, 9]'}]
retr

  9%|▉         | 9/100 [00:03<00:31,  2.92it/s]


KeyboardInterrupt: 